# stocks 정보 최신화

In [124]:
# pykrx to mysql
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from openpyxl import load_workbook
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

start = '20211015'
end = '20211018'
# end = str(datetime.now().date()).replace('-', '')
period = pd.date_range(start=start, end=end).strftime("%Y%m%d").tolist()

for date in period:
  try:
    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8')
    conn = db_connection.connect()
    df = stock.get_market_ohlcv_by_ticker(date, market="KOSPI")
    time.sleep(0.5)
    if (df['시가'][0] == 0):
      continue
    df['date'] = date
    df = df.reset_index().set_index('date').drop(['거래대금', '등락률'], axis=1).rename(columns={'티커':'ticker', '시가':'open', '고가':'high', '저가':'low', '종가':'close', '거래량':'volume'})
    df.to_sql(name='stocks_price', con=db_connection, if_exists='append', index=True, index_label="date")
    conn.close()
  except Exception as ex:
    print(ex, date)
    pass


(pymysql.err.IntegrityError) (1062, "Duplicate entry '060310-2019-01-02' for key 'PRIMARY'")
[SQL: INSERT INTO stocks_price (date, ticker, open, high, low, close, volume) VALUES (%(date)s, %(ticker)s, %(open)s, %(high)s, %(low)s, %(close)s, %(volume)s)]
[parameters: ({'date': '20190102', 'ticker': '060310', 'open': 2280, 'high': 2280, 'low': 2160, 'close': 2190, 'volume': 24598}, {'date': '20190102', 'ticker': '054620', 'open': 4225, 'high': 4290, 'low': 4155, 'close': 4205, 'volume': 23428}, {'date': '20190102', 'ticker': '265520', 'open': 21850, 'high': 22150, 'low': 21050, 'close': 21500, 'volume': 146314}, {'date': '20190102', 'ticker': '211270', 'open': 6460, 'high': 6640, 'low': 6460, 'close': 6600, 'volume': 23224}, {'date': '20190102', 'ticker': '035760', 'open': 202700, 'high': 202900, 'low': 196500, 'close': 196500, 'volume': 58208}, {'date': '20190102', 'ticker': '051500', 'open': 26350, 'high': 27250, 'low': 25500, 'close': 25550, 'volume': 54574}, {'date': '20190102', 'tic

# 볼린저밴드 구하기

In [125]:
import pandas as pd
import pymysql
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
# PW = 'root'
PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()
try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  getPeriodSql = "SELECT DISTINCT date FROM stocks_price ORDER BY date DESC"
  cursor.execute(getPeriodSql)
  period = list(cursor.fetchall())

  for ticker in tickers:
    for date in period:
      # print(ticker[0], date[0].strftime("%Y%m%d"))
      try:
        getLast20InfoSql = "SELECT * FROM stocks_price WHERE ticker = '" + ticker[0] + "' AND date <= '" + date[0].strftime("%Y%m%d") + "' ORDER BY date DESC limit 20"
        cursor.execute(getLast20InfoSql)
        last20InfoDf = pd.DataFrame(cursor.fetchall())
        if len(last20InfoDf.index) < 20:
          continue
        if last20InfoDf.iloc[0][6] == 0:
          continue
        
        last20InfoDfClose = last20InfoDf.loc[:, 5]
        avg = last20InfoDfClose.mean()
        std = last20InfoDfClose.std()
        tmpInfo = last20InfoDf.loc[0, [0, 1, 5]]
        hi = round(avg + std * 2)
        me = round(avg)
        lo = round(avg - std * 2)
        bw = (hi - lo) / me
        pos = (tmpInfo[5] - lo) / (hi - lo)

        insertBollSql = "REPLACE INTO boll(date, ticker, close, low, medium, high, bandWidth, position) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)" 
        cursor.execute(insertBollSql, (tmpInfo[0], tmpInfo[1], tmpInfo[5], lo, me, hi, bw, pos))
        db.commit()
      except Exception as ex1:
        print('ex1', ex1, ticker[0], date[0].strftime("%Y%m%d"), tmpInfo[0], tmpInfo[1], tmpInfo[5], lo, me, hi, bw, pos)
        # print(tmpInfo[0], tmpInfo[1], tmpInfo[2], hi, me, lo, bw)
        pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

# Z-SCORE
- (현재값 - 평균) / 표준편차

In [6]:
import pandas as pd
import pymysql
import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

n_days = [10, 20, 30, 90, 180]

try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  getPeriodSql = "SELECT DISTINCT date FROM stocks_price ORDER BY date DESC"
  cursor.execute(getPeriodSql)
  period = list(cursor.fetchall())

  for ticker in tickers:
    for date in period:
      for day in n_days:
        # print(ticker[0], date[0].strftime("%Y%m%d"))
        try:
          getLastNBollSql = "SELECT * FROM boll WHERE ticker = '" + ticker[0] + "' AND date <= '" + date[0].strftime("%Y%m%d") + "' ORDER BY date DESC limit " + str(day)
          cursor.execute(getLastNBollSql)
          lastNBollDf = pd.DataFrame(cursor.fetchall())
          if len(lastNBollDf.index) < day:
            continue

          lastNBollBandWidthDf = lastNBollDf.loc[:, 6]
          avg = lastNBollBandWidthDf.mean()
          std = lastNBollBandWidthDf.std()
          tmpInfo = lastNBollDf.loc[0, [0, 1, 6]]
          z_score = (tmpInfo[6] - avg) / std
          
          insertZScoreSql = "REPLACE INTO z_score(date, ticker, period, bandWidth, z_score) VALUES(%s, %s, %s, %s, %s)"
          cursor.execute(insertZScoreSql, (tmpInfo[0], tmpInfo[1], day, tmpInfo[6], z_score))
          db.commit()
        except Exception as ex1:
          print('ex1', ex1, ticker[0], date[0].strftime("%Y%m%d"))
          # print(tmpInfo[0], tmpInfo[1], tmpInfo[2], hi, me, lo, bw)
          pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

C:\Users\Jack\AppData\Local\Temp/ipykernel_19356/234392392.py:41: RuntimeWarning: invalid value encountered in double_scalars
  z_score = (tmpInfo[6] - avg) / std


ex1 (1366, "Incorrect double value: 'nan' for column `INDEX_DUCK`.`z_score`.`z_score` at row 1") 001770 20200324
ex1 (1366, "Incorrect double value: 'nan' for column `INDEX_DUCK`.`z_score`.`z_score` at row 1") 001770 20200324
ex1 (1366, "Incorrect double value: 'nan' for column `INDEX_DUCK`.`z_score`.`z_score` at row 1") 001770 20200324
ex1 (1366, "Incorrect double value: 'nan' for column `INDEX_DUCK`.`z_score`.`z_score` at row 1") 001770 20200323
ex1 (1366, "Incorrect double value: 'nan' for column `INDEX_DUCK`.`z_score`.`z_score` at row 1") 001770 20200323
ex1 (1366, "Incorrect double value: 'nan' for column `INDEX_DUCK`.`z_score`.`z_score` at row 1") 001770 20200323
ex1 (1366, "Incorrect double value: 'nan' for column `INDEX_DUCK`.`z_score`.`z_score` at row 1") 001770 20200320
ex1 (1366, "Incorrect double value: 'nan' for column `INDEX_DUCK`.`z_score`.`z_score` at row 1") 001770 20200320
ex1 (1366, "Incorrect double value: 'nan' for column `INDEX_DUCK`.`z_score`.`z_score` at row 1")

# MFI
- TP(Typical price) = (당일 고가 + 당일 저가 + 당일 종가) / 3
- RMF = TP * 당일 거래량
  - Positive RMF: 당일 TP > 전일 TP
  - Negative RMF: 당일 TP < 전일 TP
- MFI = n일간 positive RMF 합계 / (n일간 positive RMF 합계 + n일간 negative RMF 합계) * 100

In [84]:
import pandas as pd
import pymysql
import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
# PW = 'root'
PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

n_days = [10]

try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  getPeriodSql = "SELECT DISTINCT date FROM stocks_price ORDER BY date DESC"
  cursor.execute(getPeriodSql)
  period = list(cursor.fetchall())

  for ticker in tickers:
    for date in period:
      for day in n_days:
        # print(ticker[0], date[0].strftime("%Y%m%d"))
        try:
          getLast10InfoSql = "SELECT * FROM stocks_price WHERE ticker = '" + ticker[0] + "' AND date <= '" + date[0].strftime("%Y%m%d") + "' ORDER BY date DESC limit " + str(day + 1)
          cursor.execute(getLast10InfoSql)
          last10InfoDf = pd.DataFrame(cursor.fetchall())
          if len(last10InfoDf.index) < day + 1:
            continue
          tpList = []
          for idx in last10InfoDf.index:
            info = last10InfoDf.loc[idx]
            tpList.append(round((info[3] + info[4] + info[5]) / 3))
          last10InfoDf['TP'] = tpList

          positiveRMF = 0
          negativeRMF = 0
          for idx in range(0, len(last10InfoDf) - 1):
            today = last10InfoDf.iloc[idx]
            yesterday = last10InfoDf.iloc[idx + 1]
            if today['TP'] > yesterday['TP']:
              # positive
              positiveRMF += today['TP'] * today[6]
            elif today['TP'] < yesterday['TP']:
              # negative
              negativeRMF += today['TP'] * today[6]
          
          MFI = positiveRMF / (positiveRMF + negativeRMF) * 100
          todayInfo = last10InfoDf.iloc[0, [0, 1, 7]]

          insertBollSql = "REPLACE INTO mfi(date, ticker, period, tp, mfi) VALUES(%s, %s, %s, %s, %s)"
          cursor.execute(insertBollSql, (todayInfo[0], todayInfo[1], day, todayInfo['TP'], MFI))
          db.commit()
        except Exception as ex1:
          print('ex1', ex1, ticker[0], date[0].strftime("%Y%m%d"))
          pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

ex1 division by zero 000030 20211018
ex1 division by zero 000030 20211015
ex1 division by zero 000030 20211014
ex1 division by zero 000030 20211013
ex1 division by zero 000030 20211012
ex1 division by zero 000030 20211008
ex1 division by zero 000030 20211007
ex1 division by zero 000030 20211006
ex1 division by zero 000030 20211005
ex1 division by zero 000030 20211001
ex1 division by zero 000030 20210930
ex1 division by zero 000030 20210929
ex1 division by zero 000030 20210928
ex1 division by zero 000030 20210927
ex1 division by zero 000030 20210924
ex1 division by zero 000030 20210923
ex1 division by zero 000030 20210917
ex1 division by zero 000030 20210916
ex1 division by zero 000030 20210915
ex1 division by zero 000030 20210914
ex1 division by zero 000030 20210913
ex1 division by zero 000030 20210910
ex1 division by zero 000030 20210909
ex1 division by zero 000030 20210908
ex1 division by zero 000030 20210907
ex1 division by zero 000030 20210906
ex1 division by zero 000030 20210903
e

C:\Users\LGCNS\AppData\Local\Temp/ipykernel_3948/865323635.py:54: RuntimeWarning: invalid value encountered in longlong_scalars
  MFI = positiveRMF / (positiveRMF + negativeRMF) * 100


ex1 division by zero 000040 20200218
ex1 (1366, "Incorrect double value: 'nan' for column `INDEX_DUCK`.`mfi`.`mfi` at row 1") 000040 20200217
ex1 division by zero 000150 20191017
ex1 division by zero 000150 20191016
ex1 division by zero 000150 20191015
ex1 (1366, "Incorrect double value: 'nan' for column `INDEX_DUCK`.`mfi`.`mfi` at row 1") 000150 20191014
ex1 division by zero 000155 20191017
ex1 division by zero 000155 20191016
ex1 division by zero 000155 20191015
ex1 (1366, "Incorrect double value: 'nan' for column `INDEX_DUCK`.`mfi`.`mfi` at row 1") 000155 20191014
ex1 division by zero 000157 20191017
ex1 division by zero 000157 20191016
ex1 division by zero 000157 20191015
ex1 (1366, "Incorrect double value: 'nan' for column `INDEX_DUCK`.`mfi`.`mfi` at row 1") 000157 20191014
ex1 division by zero 000210 20210122
ex1 division by zero 000210 20210121
ex1 division by zero 000210 20210120
ex1 division by zero 000210 20210119
ex1 division by zero 000210 20210118
ex1 division by zero 0002

# bollinger squeeze back test

In [91]:
import pandas as pd
import pymysql
import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
# PW = 'root'
PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

## Back test 파라미터
# 기간
start = "20200101"
end = "20211001"
# 볼린저밴드 %b 매수/매도
pos_bid = 0.8
pos_ask = 0.2
# 볼린저밴드 Bandwidth
bw_bid = -1.5
# bw_ask = -1.5
# MFI
mfi_bid = 80
mfi_ask = 20
# 투자금
amount = 1000000


try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())
  getPeriodSql = "SELECT DISTINCT date FROM stocks_price WHERE date > '2020-01-01' ORDER BY date"
  cursor.execute(getPeriodSql)
  period = list(cursor.fetchall())
  history = []
  pnl = 0
  dic = {}
  for ticker in tickers:
    # print(ticker[0], date[0].strftime("%Y%m%d"))
    try:
      getBollSql = "SELECT * FROM boll WHERE ticker = '" + ticker[0] + "'  ORDER BY date"
      cursor.execute(getBollSql)
      bollDf = pd.DataFrame(cursor.fetchall()).rename(columns={0:'date', 1:'ticker', 2:'close', 3:'low', 4:'medium', 5:'high', 6:'bandWidth', 7:'position'})
      bollDf.set_index('date', drop=True, inplace=True)
      getMFISql = "SELECT * FROM mfi WHERE ticker = '" + ticker[0] + "' ORDER BY date"
      cursor.execute(getMFISql)
      mfiDf = pd.DataFrame(cursor.fetchall()).rename(columns={0:'date', 1:'ticker', 2:'period', 3:'tp', 4:'mfi'})      
      mfiDf.set_index('date', drop=True, inplace=True)
      mfiDf = mfiDf.drop(['ticker', 'period'], axis=1)
      getZScoreSql = "SELECT * FROM z_score WHERE ticker = '" + ticker[0] + "' AND period = '180' ORDER BY date"
      cursor.execute(getZScoreSql)
      zscoreDf = pd.DataFrame(cursor.fetchall()).rename(columns={0:'date', 1:'ticker', 2:'period', 3:'bandWidth', 4:'z_score'})      
      zscoreDf = zscoreDf.drop(['ticker', 'period', 'bandWidth'], axis=1)
      zscoreDf.set_index('date', drop=True, inplace=True)
      sumDf = pd.concat([bollDf, mfiDf, zscoreDf], axis=1)
      isBid = False
      bidPrice = 0
      for idx in sumDf.index:
        row = sumDf.loc[idx].copy()
        if pd.isna(row['z_score']):
          continue
        if not isBid:
          if row['position'] >= 0.8 and row['mfi'] >= 80 and row['z_score'] < -1.5:
            row['status'] = 'bid'
            history.append(row)
            isBid = True
            bidPrice = row['close']
        else:
          if row['position'] <= 0.2 or row['mfi'] <= 20:
            row['status'] = 'ask'
            history.append(row)
            isBid = False
            pnl += (1 - (bidPrice / row['close'])) * 100      
            if dic in ticker[0]:
              dic[ticker[0]].append("bid: " + str(bidPrice) + ", ask: " + str(row['close']) + ", pnl: " + str(pnl))
            else:
              dic[ticker[0]] = ["bid: " + str(bidPrice) + ", ask: " + str(row['close']) + ", pnl: " + str(pnl)]
      if isBid:
        idxLst = list(sumDf.index).sort()
        row = sumDf.loc[idxLst[-1]].copy()
        pnl += (1 - (bidPrice / row['close'])) * 100      
        if dic in ticker[0]:
          dic[ticker[0]].append("bid: " + str(bidPrice) + ", having: " + str(row['close']) + ", pnl: " + str(pnl))
        else:
          dic[ticker[0]] = ["bid: " + str(bidPrice) + ", having: " + str(row['close']) + ", pnl: " + str(pnl)]
        
      # insertBollSql = "REPLACE INTO mfi(date, ticker, period, tp, mfi) VALUES(%s, %s, %s, %s, %s)"
      # cursor.execute(insertBollSql, (todayInfo[0], todayInfo[1], day, todayInfo['TP'], MFI))
      # db.commit()
    except Exception as ex1:
      print('ex1', ex1, ticker[0])
      pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

ex1 "None of ['date'] are in the columns" 000030
ex1 "None of ['date'] are in the columns" 002250
ex1 "None of ['date'] are in the columns" 004987
ex1 "None of ['date'] are in the columns" 004989
ex1 "None of ['date'] are in the columns" 007630
ex1 "None of ['date'] are in the columns" 021820
ex1 "None of ['date'] are in the columns" 079660
ex1 "['ticker' 'period' 'bandWidth'] not found in axis" 089860
ex1 "['ticker' 'period' 'bandWidth'] not found in axis" 097520
ex1 "['ticker' 'period' 'bandWidth'] not found in axis" 137310
ex1 "['ticker' 'period' 'bandWidth'] not found in axis" 139990
ex1 "None of ['date'] are in the columns" 140890
ex1 "['ticker' 'period' 'bandWidth'] not found in axis" 178920
ex1 "['ticker' 'period' 'bandWidth'] not found in axis" 248070
ex1 "['ticker' 'period' 'bandWidth'] not found in axis" 259960
ex1 "['ticker' 'period' 'bandWidth'] not found in axis" 271940
ex1 "['ticker' 'period' 'bandWidth'] not found in axis" 302440
ex1 "['ticker' 'period' 'bandWidth'] not 

# 파라미터 최적화

In [ ]:
import itertools

param_grid = {
              'market': ['KOSPI', 'KOSDAQ'],
              'position_bid': [0.7, 0.8, 0.9],
              'position_ask': [0.1, 0.2, 0.3],
              'mfi_bid': [70, 80, 90],
              'mfi_ask': [10, 20, 30],
              'z_score': [-1.0, -1.5, -2.0]
              }

# generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
